# tRAT 3

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London

**Module:** ELEC70066 - Applied Advanced Optimisation

**Version:** 1.1.1 - 29/01/2025

---
---

# Exercise

Consider a three-state boolean linear programme

$$
\begin{array}{ll}
\displaystyle \min_{x} &  c^\top x \\
\text{s.t. } & Ax \preccurlyeq b,\\
& x_i\in \{0, 0.5, 1\}, \qquad i = 1, \dots, n.
\end{array}\tag{1}
$$

where the variable $x$ is constrained to have components equal to zero, half or one. You can think of $x_i$ as a job we can accept (1), subcontract (0.5) or decline (0), and $−c_i$ as the (positive) revenue we generate if we accept or subcontract job $i$. We can think of $Ax \preccurlyeq b$ as a set of limits on $m$ resources. $A_{ij}$, which is positive, is the amount of resource $i$ consumed if we accept job $j$; If the job is subcontracted we have half the revenues, but we also half the amount of resources spent. $b_i$, which is positive, is the amount of resource $i$ available.

For your convenience, the relaxation of the problem (i.e. $0 \le x\le 1$) is already solved in the code below.

In [1]:
import cvxpy as cp
import numpy as np

n = 100
m = 300
np.random.seed(1)
A = np.random.random((m, n)) # positve
b = np.dot(A,np.ones(n)/2) # postive
c = -np.random.random(n) # negative, so -c is positive

In [37]:
# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x),[A @ x <= b, x >= 0, x<= 1])
prob.solve()

# Print result.
L=prob.value
print("\nThe optimal value is", prob.value)
#print("A solution x is", x.value)


The optimal value is -34.58971131765026


1.   Explain how you would develop a threshold rounding for this problem.
2.   Carry out threshold rounding. For each value of of the threshold(s), note the objective value $c^\top \hat{x}$ and the maximum constraint violation $\max_i(A\hat{x} − b)_i$.
3.   Find the minimum feasible objective value and note it as the upper bound $U$. Compute the relative suboptimality percentage as $\frac{|U-L|}{|L|} \times 100$.

Part 1: 

We can use a matrix of t1 and t2 both of 50 values uniformly distributed between [0, 0.5] and [0.5, 1]. By iterating through them and finding the maximum violation and the objective value, we can find the feasible optimal $\hat x$. 

In [39]:
# Part 2
t_upper = np.linspace(0,0.5, 50)
t_lower = np.linspace(0.5,1, 50)

xi = []
objective_values = []
max_violations = []
feasible_indices = []

for t1 in t_upper:
    for t2 in t_lower:
        xi_hat = []
        for x_value in x.value:
            if x_value > t1:
                xi_hat.append(1)
            elif x_value < t2:
                xi_hat.append(0)
            else:
                xi_hat.append(0.5)
        xi.append(xi_hat)
    
        objective_values.append(c @ xi_hat)

        violation = np.max(A @ xi_hat - b)
        max_violations.append(violation)

        if max_violations[-1] <= 0:
            feasible_indices.append((t1, t2))

print('feasible indices:', feasible_indices)
print(objective_values)

feasible_obj_values = [objective_values[i] for i in range(len(objective_values)) 
                      if max_violations[i] <= 0]

print('feasible objective values: ', feasible_obj_values)
print(feasible_obj_values[-1])
# Find t values where we get the optimal objective value
optimal_indices = np.where(np.array(objective_values) == min(feasible_obj_values))[0]
optimal_t_pairs = []

for idx in optimal_indices:
    t1_idx = idx // len(t_lower)
    t2_idx = idx % len(t_lower)
    optimal_t_pairs.append((t_upper[t1_idx], t_lower[t2_idx]))

t_feasible_upper = feasible_indices[1][0]
t_feasible_lower = feasible_indices[1][1]

print(t_feasible_upper, t_feasible_lower)

# Part 3
U = min(feasible_obj_values)

relative_suboptimality = abs(U - L) / abs(L) * 100
print('Relative suboptimality:', relative_suboptimality, "%")

feasible indices: [(0.5, 0.5), (0.5, 0.5102040816326531), (0.5, 0.5204081632653061), (0.5, 0.5306122448979592), (0.5, 0.5408163265306123), (0.5, 0.5510204081632653), (0.5, 0.5612244897959183), (0.5, 0.5714285714285714), (0.5, 0.5816326530612245), (0.5, 0.5918367346938775), (0.5, 0.6020408163265306), (0.5, 0.6122448979591837), (0.5, 0.6224489795918368), (0.5, 0.6326530612244898), (0.5, 0.6428571428571428), (0.5, 0.6530612244897959), (0.5, 0.6632653061224489), (0.5, 0.673469387755102), (0.5, 0.6836734693877551), (0.5, 0.6938775510204082), (0.5, 0.7040816326530612), (0.5, 0.7142857142857143), (0.5, 0.7244897959183674), (0.5, 0.7346938775510203), (0.5, 0.7448979591836735), (0.5, 0.7551020408163265), (0.5, 0.7653061224489796), (0.5, 0.7755102040816326), (0.5, 0.7857142857142857), (0.5, 0.7959183673469388), (0.5, 0.8061224489795917), (0.5, 0.8163265306122449), (0.5, 0.8265306122448979), (0.5, 0.8367346938775511), (0.5, 0.846938775510204), (0.5, 0.8571428571428571), (0.5, 0.8673469387755102),